<a href="https://colab.research.google.com/github/felix-route10ai/computer_vision_project_for_CAV/blob/main/CAV_OSM_Feature_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install -q googlemaps polyline pandas

In [ ]:
# Imports
import os
import math
import polyline
import pandas as pd
import googlemaps

# Google API key
API_KEY = "<your google maps API key>"
gmaps = googlemaps.Client(key=API_KEY)

# === 1. Define origin/destination ===
origin = "Port of Dover, Dover CT16 1JA"
destination = "Magna Park, Plot 510, GLP, Milton Keynes MK17 7AB"

# === 2. Fetch directions from Google Maps ===
directions = gmaps.directions(origin, destination, mode="driving", units="metric")
steps = directions[0]["legs"][0]["steps"]

# === 3. Decode polylines from each step ===
segments = []
for i, step in enumerate(steps):
    points = polyline.decode(step["polyline"]["points"])
    for lat, lng in points:
        segments.append({
            "segment_id": i + 1,
            "instruction": step.get("html_instructions", ""),
            "lat": lat,
            "lng": lng
        })

df_route = pd.DataFrame(segments)

# === 4. Save full route to Google Drive ===
output_dir = "<your output directory>"
os.makedirs(output_dir, exist_ok=True)

df_route.to_csv(f"{output_dir}/route_segments.csv", index=False)
print(f"route_segments.csv saved with {len(df_route)} rows")

# === Helper: Haversine Distance (in km) ===
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * \
        math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    return R * 2 * math.asin(math.sqrt(a))

# === Resample the route every X meters ===
def resample_route(df_route, spacing_m=50):
    resampled = [df_route.iloc[0]]
    accum_distance = 0
    last_lat, last_lng = df_route.iloc[0]["lat"], df_route.iloc[0]["lng"]

    for _, row in df_route.iterrows():
        dist = haversine(last_lat, last_lng, row["lat"], row["lng"]) * 1000  # km to meters
        accum_distance += dist
        if accum_distance >= spacing_m:
            resampled.append(row)
            last_lat, last_lng = row["lat"], row["lng"]
            accum_distance = 0

    df_resampled = pd.DataFrame(resampled).reset_index(drop=True)
    df_resampled["segment_id"] = df_resampled.index + 1
    return df_resampled

# === Call it with 50m spacing ===
df_resampled_50m = resample_route(df_route, spacing_m=50)
df_resampled_50m.to_csv("<your output directory>/df_resampled_50m.csv", index=False)
print("Saved df_resampled_50m.csv with", len(df_resampled_50m), "segments.")
df_resampled_50m.head()

route_segments.csv saved with 10558 rows
Saved df_resampled_50m.csv with 4257 segments.


,segment_id,instruction,lat,lng
0,1,Head <b>northeast</b> on <b>Waterloo Cres</b> ...,51.12090,1.31345
1,2,Head <b>northeast</b> on <b>Waterloo Cres</b> ...,51.12131,1.31378
2,3,Head <b>northeast</b> on <b>Waterloo Cres</b> ...,51.12162,1.31408
3,4,Head <b>northeast</b> on <b>Waterloo Cres</b> ...,51.12197,1.31442
4,5,Head <b>northeast</b> on <b>Waterloo Cres</b> ...,51.12239,1.31494


In [ ]:
!pip install googlemaps polyline geopandas shapely

In [ ]:
# CAV Risk POI Extractor (Google Maps Focused)

# This notebook will:
# 1. Load route segments from a CSV (lat/lng)
# 2. Use Google Maps Directions API to get route polyline
# 3. Use Google Roads API to snap points and enrich with road geometry
# 4. Apply rules to detect features (lane merges, curves, etc.)
# 5. Save POIs with lat/lng and type to Drive for image download

import pandas as pd
import geopandas as gpd
import googlemaps
import polyline
import time
from shapely.geometry import LineString, Point
import os

# === CONFIG ===
API_KEY = "<your google maps API key>"
ROUTE_CSV = "<your output directory>/df_resampled_50m.csv"
OUTPUT_POIS_CSV = "<your output directory>/route_pois.csv"
MAX_WAYPOINTS_PER_REQUEST = 23  # Google API limit is 25 (start + end + 23)
REQUEST_DELAY_SEC = 2

# === INIT ===
gmaps = googlemaps.Client(key=API_KEY)

# === 1. Load Route ===
df_route = pd.read_csv(ROUTE_CSV)
waypoints = df_route[["lat", "lng"]].values.tolist()

# === 2. Chunked API Calls ===
def fetch_directions_in_chunks(points):
    all_snapped = []
    for i in range(0, len(points) - 1, MAX_WAYPOINTS_PER_REQUEST):
        chunk_start = points[i]
        chunk_end = points[min(i + MAX_WAYPOINTS_PER_REQUEST, len(points) - 1)]
        chunk_via = points[i + 1:min(i + MAX_WAYPOINTS_PER_REQUEST, len(points) - 1)]

        print(f"Fetching segment {i} to {i + MAX_WAYPOINTS_PER_REQUEST}...")
        try:
            directions = gmaps.directions(origin=chunk_start, destination=chunk_end, waypoints=chunk_via, mode="driving")
            polyline_str = directions[0]["overview_polyline"]["points"]
            snapped_points = polyline.decode(polyline_str)
            all_snapped.extend(snapped_points)
        except Exception as e:
            print(f"Error fetching directions for segment {i}: {e}")

        time.sleep(REQUEST_DELAY_SEC)

    return all_snapped

# === 3. Get full snapped route ===
snapped_points = fetch_directions_in_chunks(waypoints)

# === 4. Feature Detection Rules ===
def calculate_bearing(p1, p2):
    import math
    lat1, lon1, lat2, lon2 = map(math.radians, [p1[0], p1[1], p2[0], p2[1]])
    dlon = lon2 - lon1
    y = math.sin(dlon) * math.cos(lat2)
    x = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(dlon)
    bearing = math.atan2(y, x)
    return (math.degrees(bearing) + 360) % 360

def distance(p1, p2):
    from geopy.distance import geodesic
    return geodesic(p1, p2).meters

poi_records = []
for i in range(2, len(snapped_points) - 2):
    prev = snapped_points[i - 1]
    curr = snapped_points[i]
    next_ = snapped_points[i + 1]

    # === CURVE OR JUNCTION ===
    angle = abs(calculate_bearing(prev, curr) - calculate_bearing(curr, next_))
    if angle > 45:
        poi_records.append({"lat": curr[0], "lng": curr[1], "feature": "curve_or_junction"})

    # === LANE MERGE HEURISTIC ===
    d1 = distance(prev, curr)
    d2 = distance(curr, next_)
    d_change = abs(d2 - d1)
    if d_change > 10 and angle < 20:
        poi_records.append({"lat": curr[0], "lng": curr[1], "feature": "possible_lane_merge"})

# === 5. Save POIs ===
df_pois = pd.DataFrame(poi_records)
df_pois.to_csv(OUTPUT_POIS_CSV, index=False)

print(f"Saved {len(df_pois)} POIs to: {OUTPUT_POIS_CSV}")
df_pois.head()


Fetching segment 0 to 23...
Fetching segment 23 to 46...
Fetching segment 46 to 69...
Fetching segment 69 to 92...
Fetching segment 92 to 115...
Fetching segment 115 to 138...
Fetching segment 138 to 161...
Fetching segment 161 to 184...
Fetching segment 184 to 207...
Fetching segment 207 to 230...
Fetching segment 230 to 253...
Fetching segment 253 to 276...
Fetching segment 276 to 299...
Fetching segment 299 to 322...
Fetching segment 322 to 345...
Fetching segment 345 to 368...
Fetching segment 368 to 391...
Fetching segment 391 to 414...
Fetching segment 414 to 437...
Fetching segment 437 to 460...
Fetching segment 460 to 483...
Fetching segment 483 to 506...
Fetching segment 506 to 529...
Fetching segment 529 to 552...
Fetching segment 552 to 575...
Fetching segment 575 to 598...
Fetching segment 598 to 621...
Fetching segment 621 to 644...
Fetching segment 644 to 667...
Fetching segment 667 to 690...
Fetching segment 690 to 713...
Fetching segment 713 to 736...
Fetching segment 7

,lat,lng,feature
0,51.12162,1.31408,possible_lane_merge
1,51.12223,1.31474,possible_lane_merge
2,51.12256,1.31520,possible_lane_merge
3,51.12290,1.31576,possible_lane_merge
4,51.12329,1.31649,curve_or_junction


In [ ]:
!pip install -q pillow

In [ ]:
# POI Image Downloader

!pip install pillow requests

import os
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import math
import time

# === CONFIG ===
API_KEY = "<your google maps API key>"
POIS_CSV = "<your output directory>/route_pois.csv"
OUTPUT_DIR = "<your output directory>/poi_images"

# === HELPER FUNCTIONS ===
def calculate_heading(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = map(math.radians, [lat1, lng1, lat2, lng2])
    dlon = lng2 - lng1
    y = math.sin(dlon) * math.cos(lat2)
    x = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(dlon)
    bearing = math.degrees(math.atan2(y, x))
    return (bearing + 360) % 360

def optimized_download(df, api_key, output_dir, start_index=0, max_images=None):
    os.makedirs(output_dir, exist_ok=True)
    base_url = "https://maps.googleapis.com/maps/api/streetview"

    DELAY_BETWEEN_REQUESTS = 2
    DELAY_AFTER_ERROR = 15
    MAX_RETRIES = 3

    image_records = []
    error_log = []

    end_index = len(df) if not max_images else min(start_index + max_images, len(df))

    print(f"Optimized download starting...")
    print(f"Processing rows {start_index} to {end_index-1} ({end_index-start_index} images)")
    print(f"This will take ~{(end_index-start_index) * DELAY_BETWEEN_REQUESTS / 60:.1f} minutes")
    print("=" * 60)

    for i in range(start_index, end_index):
        row = df.iloc[i]
        lat = row["lat"]
        lng = row["lng"]
        feature = row["feature"]
        segment_id = i + 1  # or use a unique ID field

        location = f"{lat},{lng}"
        image_path = f"{output_dir}/poi_{segment_id:04d}_{feature}.jpg"

        heading = 0
        if i < len(df) - 1:
            next_row = df.iloc[i + 1]
            heading = calculate_heading(lat, lng, next_row["lat"], next_row["lng"])

        if os.path.exists(image_path):
            print(f"[{i}] Exists - Skipping")
            image_records.append({"poi_id": segment_id, "lat": lat, "lng": lng, "feature": feature, "image_path": image_path})
            continue

        params = {
            "size": "640x640",
            "location": location,
            "fov": 90,
            "pitch": 0,
            "heading": heading,
            "key": api_key
        }

        success = False
        for attempt in range(MAX_RETRIES):
            try:
                response = requests.get(base_url, params=params, timeout=30)
                if response.status_code == 200 and 'image' in response.headers.get('content-type', ''):
                    img = Image.open(BytesIO(response.content))
                    img.save(image_path)
                    image_records.append({"poi_id": segment_id, "lat": lat, "lng": lng, "feature": feature, "image_path": image_path})
                    print(f"[{i}] Saved {os.path.basename(image_path)}")
                    success = True
                    break
                else:
                    print(f"[{i}] Invalid content or status code: {response.status_code}")
                    time.sleep(DELAY_AFTER_ERROR)
            except Exception as e:
                print(f"[{i}] Exception: {e}")
                time.sleep(DELAY_AFTER_ERROR)

        if not success:
            error_log.append({"poi_id": segment_id, "lat": lat, "lng": lng, "feature": feature, "error": "Failed after retries"})

        if i < end_index - 1:
            time.sleep(DELAY_BETWEEN_REQUESTS)

    df_images = pd.DataFrame(image_records)
    df_images.to_csv(f"{output_dir}/streetview_metadata.csv", index=False)
    if error_log:
        pd.DataFrame(error_log).to_csv(f"{output_dir}/error_log.csv", index=False)

    print(f"Downloaded {len(image_records)} images, {len(error_log)} errors logged.")
    return df_images, error_log

# === USAGE ===
df = pd.read_csv(POIS_CSV)
images, errors = optimized_download(df, API_KEY, OUTPUT_DIR)

Optimized download starting...
Processing rows 0 to 3097 (3098 images)
This will take ~103.3 minutes
[0] ✅ Saved poi_0001_possible_lane_merge.jpg
[1] ✅ Saved poi_0002_possible_lane_merge.jpg
[2] ✅ Saved poi_0003_possible_lane_merge.jpg
[3] ✅ Saved poi_0004_possible_lane_merge.jpg
[4] ✅ Saved poi_0005_curve_or_junction.jpg
[5] ✅ Saved poi_0006_curve_or_junction.jpg
[6] ✅ Saved poi_0007_possible_lane_merge.jpg
[7] ✅ Saved poi_0008_possible_lane_merge.jpg
[8] ✅ Saved poi_0009_possible_lane_merge.jpg
[9] ✅ Saved poi_0010_possible_lane_merge.jpg
[10] ✅ Saved poi_0011_possible_lane_merge.jpg
[11] ✅ Saved poi_0012_possible_lane_merge.jpg
[12] ✅ Saved poi_0013_curve_or_junction.jpg
[13] ✅ Saved poi_0014_curve_or_junction.jpg
[14] ✅ Saved poi_0015_possible_lane_merge.jpg
[15] ✅ Saved poi_0016_possible_lane_merge.jpg
[16] ✅ Saved poi_0017_possible_lane_merge.jpg
[17] ✅ Saved poi_0018_possible_lane_merge.jpg
[18] ✅ Saved poi_0019_possible_lane_merge.jpg
[19] ✅ Saved poi_0020_possible_lane_merge.j

In [ ]:
#Create a traning data set for manual annotation

import os
import pandas as pd
import shutil

# Load metadata
metadata_path = "<your output directory>/streetview_metadata.csv"
df = pd.read_csv(metadata_path)

# Make sure paths are strings (in case they're float NaNs)
df["image_path"] = df["image_path"].astype(str)

# Target output folder
output_dir = "<your output directory>/annotation_sample_images"
os.makedirs(output_dir, exist_ok=True)

# Sample 200 images (stratified by feature, capped if needed)
sampled_dfs = []
samples_per_class = max(1, 200 // df["feature"].nunique())

for feature, group in df.groupby("feature"):
    sampled = group.sample(n=min(len(group), samples_per_class), random_state=42)
    sampled_dfs.append(sampled)

df_sampled = pd.concat(sampled_dfs).reset_index(drop=True)

# Copy selected images to annotation folder
copied = 0
for _, row in df_sampled.iterrows():
    src = row["image_path"]
    if os.path.exists(src):
        fname = os.path.basename(src)
        dst = os.path.join(output_dir, fname)
        shutil.copy(src, dst)
        copied += 1

print(f"Sampled {len(df_sampled)} images")
print(f"Copied {copied} images to: {output_dir}")

Sampled 200 images
Copied 200 images to: /content/drive/MyDrive/CAV/annotation_sample_images


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")
model.train(data="<your output directory>/data.yaml", epochs=50, imgsz=640)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Ultralytics 8.3.172 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/CAV/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pr

Overriding model.yaml nc=80 with nc=21

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

Model summary: 129 layers, 3,014,943 parameters, 3,014,927 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.5±0.2 ms, read: 0.3±0.2 MB/s, size: 92.6 KB)


...

                   all         36         85      0.803      0.233      0.273      0.155



50 epochs completed in 0.040 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.172 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,009,743 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.25it/s]


                   all         36         85      0.699       0.26      0.287      0.179
                bridge          3          4          1          0          0          0
     construction_zone          7          7          1          0     0.0136    0.00502
                 curve          1          1          0          0          0          0
         hard_shoulder         12         12      0.708      0.417      0.561      0.339
              junction          5          7          1          0    0.00507   0.000685
            lane_merge          4          4          0          0     0.0272     0.0123
      motorway_signage          3          3          1          0    0.00887    0.00292
         mutiple_lanes         27         27       0.73      0.815      0.772      0.421
      no_hard_shoulder          5          5      0.685      0.445      0.572      0.487
      obscured_signage          2          2          1          0          0          0
            roundabou

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  2,  3,  5,  6,  7,  8,  9, 10, 11, 15, 16, 17, 18, 19])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a98d48ec150>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,   

In [ ]:
#Run Inference on using Traning Data and Full Dataset

#Imports and Paths
import os
import pandas as pd
from ultralytics import YOLO
from tqdm import tqdm
import shutil

#Paths
MODEL_PATH = "<your model path>/best.pt"
IMAGE_DIR = "<your image directory/poi_images"
OUTPUT_DIR = "<your output directory>/inference_results"
CSV_OUTPUT = f"{OUTPUT_DIR}/detection_summary.csv"

# Ensure output directory is clean
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

#Load model
model = YOLO(MODEL_PATH)

#Run inference
image_files = [os.path.join(IMAGE_DIR, f) for f in os.listdir(IMAGE_DIR) if f.endswith(('.jpg', '.png'))]

print(f"Running inference on {len(image_files)} images...")

results_summary = []

for img_path in tqdm(image_files):
    result = model.predict(img_path, conf=0.4, save=True, save_txt=False, save_crop=False, project=OUTPUT_DIR, name="pred", exist_ok=True)
    dets = result[0].boxes
    if dets is not None and dets.cls.numel() > 0:
        for cls, conf in zip(dets.cls.cpu().numpy(), dets.conf.cpu().numpy()):
            results_summary.append({
                "image_file": os.path.basename(img_path),
                "class_id": int(cls),
                "confidence": float(conf)
            })
    else:
        results_summary.append({
            "image_file": os.path.basename(img_path),
            "class_id": None,
            "confidence": None
        })

#Save CSV Summary
df_results = pd.DataFrame(results_summary)
df_results.to_csv(CSV_OUTPUT, index=False)
print(f"\n Inference complete! Results saved to: {CSV_OUTPUT}")

Running inference on 3098 images...


  0%|          | 0/3098 [00:00<?, ?it/s]


image 1/1 /content/drive/MyDrive/CAV/poi_images/poi_2100_possible_lane_merge.jpg: 640x640 1 mutiple_lanes, 8.3ms
Speed: 2.3ms preprocess, 8.3ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /content/drive/MyDrive/CAV/inference_results/pred


  0%|          | 1/3098 [00:00<49:20,  1.05it/s]


image 1/1 /content/drive/MyDrive/CAV/poi_images/poi_2101_possible_lane_merge.jpg: 640x640 1 mutiple_lanes, 8.9ms
Speed: 3.2ms preprocess, 8.9ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /content/drive/MyDrive/CAV/inference_results/pred


...


image 1/1 /content/drive/MyDrive/CAV/poi_images/poi_0094_possible_lane_merge.jpg: 640x640 1 mutiple_lanes, 8.8ms
Speed: 2.6ms preprocess, 8.8ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /content/drive/MyDrive/CAV/inference_results/pred

image 1/1 /content/drive/MyDrive/CAV/poi_images/poi_0095_possible_lane_merge.jpg: 640x640 1 mutiple_lanes, 9.0ms
Speed: 2.9ms preprocess, 9.0ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /content/drive/MyDrive/CAV/inference_results/pred

image 1/1 /content/drive/MyDrive/CAV/poi_images/poi_0096_curve_or_junction.jpg: 640x640 (no detections), 9.2ms
Speed: 2.4ms preprocess, 9.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /content/drive/MyDrive/CAV/inference_results/pred


100%|█████████▉| 3095/3098 [04:02<00:00, 24.98it/s]


image 1/1 /content/drive/MyDrive/CAV/poi_images/poi_0097_curve_or_junction.jpg: 640x640 1 mutiple_lanes, 18.0ms
Speed: 2.4ms preprocess, 18.0ms inference, 4.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /content/drive/MyDrive/CAV/inference_results/pred

image 1/1 /content/drive/MyDrive/CAV/poi_images/poi_0098_possible_lane_merge.jpg: 640x640 1 mutiple_lanes, 11.6ms
Speed: 2.9ms preprocess, 11.6ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /content/drive/MyDrive/CAV/inference_results/pred

image 1/1 /content/drive/MyDrive/CAV/poi_images/poi_0099_possible_lane_merge.jpg: 640x640 1 mutiple_lanes, 8.7ms
Speed: 2.5ms preprocess, 8.7ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /content/drive/MyDrive/CAV/inference_results/pred


100%|██████████| 3098/3098 [04:02<00:00, 12.77it/s]


 Inference complete! Results saved to: /content/drive/MyDrive/CAV/inference_results/detection_summary.csv


In [ ]:
#Inference metadataset

import os
import pandas as pd
from PIL import Image
from ultralytics import YOLO

#CONFIG
PRED_IMAGE_DIR = "<your directory>/predict"
ORIGINAL_METADATA_CSV = "<your directory>/streetview_metadata.csv"
LABELS_DIR = os.path.join(PRED_IMAGE_DIR, "labels")  # YOLO stores txt labels here
ANNOTATED_IMAGES_DIR = PRED_IMAGE_DIR               # Annotated image outputs
OUTPUT_CSV = "<your directory>/inference_metadata.csv"

#Load original metadata
df_meta = pd.read_csv(ORIGINAL_METADATA_CSV)
df_meta["filename"] = df_meta["image_path"].apply(lambda x: os.path.basename(x))

#Load class map from YOLO model (use your trained model)
model = YOLO("<your model path>/best.pt")  # adjust if needed
class_map = model.names  # e.g. {0: 'roundabout', 1: 'tunnel', ...}

#Collect predictions
records = []
for i, row in df_meta.iterrows():
    filename = row["filename"]
    label_file = os.path.join(LABELS_DIR, filename.replace(".jpg", ".txt"))
    annotated_path = os.path.join(ANNOTATED_IMAGES_DIR, filename)

    if not os.path.exists(label_file):
        continue

    with open(label_file, "r") as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            if len(parts) < 5:
                continue  # Skip invalid lines

            class_id = int(float(parts[0]))
            conf = float(parts[5]) if len(parts) > 5 else None

            records.append({
                "lat": row["lat"],
                "lng": row["lng"],
                "feature": class_map[class_id],
                "confidence": conf,
                "image_path": annotated_path
            })

# === Save metadata
df_output = pd.DataFrame(records)
df_output.to_csv(OUTPUT_CSV, index=False)
print(f"Inference metadata saved: {OUTPUT_CSV}")
df_output.head()

✅ Inference metadata saved: /content/drive/MyDrive/CAV/inference_metadata.csv


""


In [ ]:
#Join data files for use with a Map - Option Streamlit used in this project

import pandas as pd
import os
from ultralytics import YOLO

#Paths
DETECTION_CSV = "<your storage directory>/detection_summary.csv"
ORIGINAL_META = "<your image directory>/streetview_metadata.csv"
YOLO_MODEL_PATH = "<your model directory>/best.pt"
OUTPUT_CSV = "<directory>/streamlit_metadata.csv"

#Load data
df_det = pd.read_csv(DETECTION_CSV)
df_meta = pd.read_csv(ORIGINAL_META)
df_meta["image_file"] = df_meta["image_path"].apply(lambda x: os.path.basename(x))

#Drop rows with missing class_id
df_det_clean = df_det.dropna(subset=["class_id"]).copy()

#Load YOLO class map
model = YOLO(YOLO_MODEL_PATH)
class_map = model.names  # e.g. {0: 'roundabout', 1: 'lane_merge', ...}

#Map class_id to labels
df_det_clean["class_id"] = df_det_clean["class_id"].astype(int)
df_det_clean["feature"] = df_det_clean["class_id"].map(class_map)

#Merge with original lat/lng + image_path
df_joined = df_det_clean.merge(df_meta[["image_file", "lat", "lng", "image_path"]], on="image_file", how="left")

#Reorder and save
df_final = df_joined[["lat", "lng", "feature", "confidence", "image_path"]]
df_final.to_csv(OUTPUT_CSV, index=False)

print(f"Cleaned Streamlit metadata saved to: {OUTPUT_CSV}")
df_final.head()

✅ Cleaned Streamlit metadata saved to: /content/drive/MyDrive/CAV/inference_results/streamlit_metadata.csv


,lat,lng,feature,confidence,image_path
0,51.68913,0.08444,mutiple_lanes,0.873651,/content/drive/MyDrive/CAV/poi_images/poi_2100...
1,51.68856,0.07867,mutiple_lanes,0.773077,/content/drive/MyDrive/CAV/poi_images/poi_2101...
2,51.68856,0.07214,mutiple_lanes,0.704700,/content/drive/MyDrive/CAV/poi_images/poi_2102...
3,51.68774,0.06428,mutiple_lanes,0.879521,/content/drive/MyDrive/CAV/poi_images/poi_2103...
4,51.68774,0.06428,hard_shoulder,0.447249,/content/drive/MyDrive/CAV/poi_images/poi_2103...


In [ ]:
#Streamlit Map Visualisation for CAV Risk Detection

# Install dependencies
!pip install streamlit folium pandas geopandas -q

import streamlit as st
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from streamlit_folium import st_folium
from PIL import Image
import os

#CONFIG
ROUTE_CSV = "<your directory>/df_resampled_50m.csv"
INFERENCE_CSV = "<your directory>/inference_results.csv"  # Must contain: lat, lng, class_name, image_path

#Evalution criteria - probably needs to be changed
CRITICAL = ["tunnel", "roundabout", "lane_merge", "construction_zone"]
MEDIUM = ["junction", "curve", "pedestrian_crossing", "obscured_signage", "multiple_lanes"]
LOW = ["bus_stop", "signage", "no_hard_shoulder", "residential", "urban", "parked_cars", "foggy"]

RISK_COLOR = {
    "CRITICAL": "red",
    "MEDIUM": "orange",
    "LOW": "green"
}

# === LOAD DATA ===
df_route = pd.read_csv(ROUTE_CSV)
df_infer = pd.read_csv(INFERENCE_CSV)

# === CLASSIFY RISK ===
def get_risk_level(cls):
    if cls in CRITICAL:
        return "CRITICAL"
    elif cls in MEDIUM:
        return "MEDIUM"
    else:
        return "LOW"

df_infer["risk"] = df_infer["class_name"].apply(get_risk_level)

# === STREAMLIT UI ===
st.set_page_config(layout="wide")
st.title("CAV Route Risk Visualisation")

features = st.multiselect("Select features to display:", options=sorted(df_infer["class_name"].unique()), default=None)

# === CREATE MAP ===
center_lat = df_route["lat"].mean()
center_lng = df_route["lng"].mean()
map_ = folium.Map(location=[center_lat, center_lng], zoom_start=9)

# Draw route in black
coords = df_route[["lat", "lng"]].values.tolist()
folium.PolyLine(coords, color="black", weight=3).add_to(map_)

# Filter by selected features
if features:
    df_show = df_infer[df_infer["class_name"].isin(features)]
else:
    df_show = df_infer

# Add POI markers
cluster = MarkerCluster().add_to(map_)
for _, row in df_show.iterrows():
    popup_html = f"""
    <b>Class:</b> {row['class_name']}<br>
    <b>Risk:</b> {row['risk']}<br>
    <img src="file://{row['image_path']}" width="300">
    """
    folium.Marker(
        location=[row["lat"], row["lng"]],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(color=RISK_COLOR[row["risk"]])
    ).add_to(cluster)

# Render map
st_data = st_folium(map_, width=1400, height=700)